In [ ]:
%pip install opencv-python
import cv2

def take_photo(filename='photo.jpg'):
    # Start webcam
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Cannot open webcam")
        return None

    print("Webcam is running. Press 's' to save photo or 'q' to quit.")

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame")
            break

        # Display the resulting frame
        cv2.imshow('Live Feed - Press s to Save or q to Quit', frame)

        key = cv2.waitKey(1)

        if key == ord('s'):  # Press 's' to save photo
            cv2.imwrite(filename, frame)
            print(f"Photo saved as {filename}")
            cv2.destroyAllWindows()  # Close the camera window
            break
        elif key == ord('q'):  # Press 'q' to quit without saving
            print("Quit without saving")
            cv2.destroyAllWindows()  # Close the camera window
            break

    # When everything is done, release the capture
    cap.release()
    return filename


# Run the capture function
try:
    filename = take_photo()
    if filename and os.path.exists(filename):
        print(f"Saved to {filename}")
        import os
        os.system(f"open {filename}")  # Mac image viewer
    else:
        print("No photo was saved.")
except Exception as e:
    print("Error:", e)


In [ ]:
# Takes two images and compares the faces, if the images have the same faces then it 
# returns that it is a match, if its different faces then it returns that it is not a match

%pip install opencv-python
import cv2

def compare_faces(image1_path, image2_path):
    # Load the images
    image1 = cv2.imread(image1_path)
    image2 = cv2.imread(image2_path)

    # Convert the images to grayscale
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    # Initialize the face detector
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Detect faces in the first image
    faces1 = face_cascade.detectMultiScale(gray1, scaleFactor=1.1, minNeighbors=5)
    faces2 = face_cascade.detectMultiScale(gray2, scaleFactor=1.1, minNeighbors=5)

    if len(faces1) == 0 or len(faces2) == 0:
        print("No faces found in one or both images.")
        return False

    # Compare the first detected face in each image
    (x1, y1, w1, h1) = faces1[0]
    (x2, y2, w2, h2) = faces2[0]

    face1 = gray1[y1:y1+h1, x1:x1+w1]
    face2 = gray2[y2:y2+h2, x2:x2+w2]

    # Resize the faces to the same size for comparison
    face1_resized = cv2.resize(face1, (w1, h1))
    face2_resized = cv2.resize(face2, (w1, h1))

    # Calculate the difference between the two faces
    difference = cv2.absdiff(face1_resized, face2_resized)
    
    # Convert to grayscale and threshold
    _, thresh = cv2.threshold(difference, 30, 255, cv2.THRESH_BINARY)

    # Count non-zero pixels in the thresholded image
    non_zero_count = cv2.countNonZero(thresh)

    # If the number of different pixels is small enough, consider it a match
    if non_zero_count < 1000:  # Adjust this threshold as needed
        print("The faces match!")
        return True
    else:
        print("The faces do not match.")
        return False

In [ ]:
import face_recognition

def compare_faces(img1_path, img2_path):
    # Load the images
    image1 = face_recognition.load_image_file(img1_path)
    image2 = face_recognition.load_image_file(img2_path)

    # Get face encodings
    encoding1 = face_recognition.face_encodings(image1)
    encoding2 = face_recognition.face_encodings(image2)

    if not encoding1:
        print(f"No face found in {img1_path}")
        return
    if not encoding2:
        print(f"No face found in {img2_path}")
        return

    # Compare faces
    match = face_recognition.compare_faces([encoding1[0]], encoding2[0], tolerance=0.6)

    if match[0]:
        print("Match: Same person")
    else:
        print("No match: Different people")


In [ ]:
compare_faces("known_faces/person1.png", "known_faces/test.png")

In [ ]:
from PIL import Image
from IPython.display import display

img1 = Image.open("known_faces/person1.jpg")
img2 = Image.open("known_faces/test.jpg")

print("Known Image:")
display(img1)

print("Test Image:")
display(img2)

In [ ]:
%pip install requests
%pip install requests pillow face-recognition

In [ ]:
import requests
import face_recognition
from io import BytesIO
import numpy as np
from PIL import Image

def preprocess_image(image):
    # Check and print if the image is in RGB mode
    print(f"Preprocessing image. Current mode: {image.mode}")
    if image.mode != 'RGB':
        print("Image is not in RGB mode. Converting to RGB...")
        image = image.convert('RGB')
    else:
        print("Image is already in RGB mode.")

    # Check for alpha channel and remove it
    if image.mode == 'RGBA':
        print("Image has an alpha channel. Removing alpha channel...")
        image = image.convert('RGB')

    # Convert to NumPy array
    img_array = np.array(image, dtype=np.uint8)

    # Ensure C-contiguous memory layout
    img_array = np.ascontiguousarray(img_array)

    # Debug: Print image properties
    print(f"Processed image properties:")
    print(f"- Shape: {img_array.shape}")
    print(f"- Dtype: {img_array.dtype}")
    print(f"- Min value: {img_array.min()}, Max value: {img_array.max()}")
    print(f"- C_CONTIGUOUS: {img_array.flags['C_CONTIGUOUS']}")

    # Save the processed image for inspection
    Image.fromarray(img_array).save("processed_image.jpg")
    print("Processed image saved as 'processed_image.jpg'.")

    return img_array

def compare_faces_from_urls(url1, url2):
    try:
        def load_image_from_url(url):
            response = requests.get(url, timeout=10)
            response.raise_for_status()

            # Open with PIL
            img = Image.open(BytesIO(response.content))

            # Preprocess the image
            return preprocess_image(img)

        # Load and preprocess both images
        image1 = load_image_from_url(url1)
        image2 = load_image_from_url(url2)

        # Encode the faces
        encodings1 = face_recognition.face_encodings(image1)
        encodings2 = face_recognition.face_encodings(image2)

        # Check if any faces were detected
        if len(encodings1) == 0 or len(encodings2) == 0:
            print("No faces detected in one or both images.")
            return False

        # Compare the first face in each image
        match = face_recognition.compare_faces([encodings1[0]], encodings2[0])

        if match[0]:
            print("The faces match!")
            return True
        else:
            print("The faces do not match.")
            return False

    except Exception as e:
        print(f"Error: {e}")
        return False

In [ ]:
compare_faces_from_urls(
    "https://ooiszpalwebsurwuuvjn.supabase.co/storage/v1/object/public/patient-images/patient-images/daaf74d2-d950-41b0-9445-05141b4e5818.jpg",
    "https://ooiszpalwebsurwuuvjn.supabase.co/storage/v1/object/public/patient-images/patient-images/daaf74d2-d950-41b0-9445-05141b4e5818.jpg"
)

In [ ]:
import cv2

def take_photo(filename='photo.jpg'):
    # Start webcam
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Cannot open webcam")
        return None

    print("Webcam is running. Press 's' to save photo or 'q' to quit.")

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame")
            break

        # Display the resulting frame
        cv2.imshow('Live Feed - Press s to Save or q to Quit', frame)

        key = cv2.waitKey(1)

        if key == ord('s'):  # Press 's' to save photo
            cv2.imwrite(filename, frame)
            print(f"Photo saved as {filename}")
            cv2.destroyAllWindows()  # Close the camera window
            break
        elif key == ord('q'):  # Press 'q' to quit without saving
            print("Quit without saving")
            cv2.destroyAllWindows()  # Close the camera window
            break

    # When everything is done, release the capture
    cap.release()
    return filename


# Run the capture function
try:
    filename = take_photo()
    if filename and os.path.exists(filename):
        print(f"Saved to {filename}")
        import os
        os.system(f"open {filename}")  # Mac image viewer
    else:
        print("No photo was saved.")
except Exception as e:
    print("Error:", e)


In [ ]:
from deepface import DeepFace

def compare_faces_from_urls(url1, url2):
    try:
        print("Comparing faces...")
        
        result = DeepFace.verify(
            img1_path=url1,
            img2_path=url2,
            model_name='Facenet',           
            detector_backend='opencv',      
            enforce_detection=True
        )

        if result["verified"]:
            print("Match: The faces belong to the same person.")
            return True
        else:
            print("No Match: The faces belong to different people.")
            return False

    except Exception as e:
        print(f"Error during face comparison: {e}")
        return False


In [ ]:
compare_faces_from_urls(
    "https://ooiszpalwebsurwuuvjn.supabase.co/storage/v1/object/public/patient-images/patient-images/daaf74d2-d950-41b0-9445-05141b4e5818.jpg",
    "https://static.wikia.nocookie.net/marvelmovies/images/2/2f/Tom_Holland.jpg/revision/latest?cb=20180221210317"
)